# Carga de datos

Se utilizan los archivos Excel con los resultados preliminares de las
elecciones parlamentarias de 2025 en Chile, entregados por el
[Servel](https://elecciones.servel.cl/).

A cada candidato se le agrega el pacto al cual pertenece y el distrito por el
cual está compitiendo, guardando la información en `datos_2025.csv`.

In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))

In [2]:
import numpy as np
import pandas as pd
from openpyxl import load_workbook
import re

from data_load_utils import add_percentage

In [3]:
# function that receives a candidate row, and returns the pact to which the
# candidate belongs (the first row above the candidate that doesn't have a 
# candidate name)
def get_pact(row, df):
    
    if pd.isna(row["candidate"]):
        return None
    
    for i in range(row.name, -1, -1):
        
        if pd.isna(df.loc[i]["candidate"]):
            return df.loc[i]["first_pact"]
        
    return None

In [4]:
def clean_party(party: str):
    """
    Quita la diferencia entre postulantes independientes asociados a un partido
    y militantes del partido (pues se tratan de igual forma en la elección).
    """
    return party.replace("IND - ", "")

def clean_candidate(candidate: str):
    """
    Quita el número antes del nombre del candidato.
    """
    return re.sub(r"^[^ ]*", "", candidate)

In [5]:
results = pd.DataFrame(
    np.empty((0, 5)),
    columns=["candidate", "pact", "party", "district", "votes"]
)

checked_tables_districts = []
total_tables_districts = []
percentage_tables_districts = []

# for each district
for n in range(1, 29):
    
    # load excel
    district_df = pd.read_excel(
        f"datos_2025/distrito_{n}.xlsx",
        header=10,
        names=["first_pact", "candidate", "party", "votes"],
        usecols="A:D"
    )

    # assign pact and district to each candidate
    district_df["pact"] = district_df.apply(
        get_pact, args=(district_df,), axis=1
    )
    district_df["district"] = n

    # remove pact rows and reorder columns
    district_df = district_df[~district_df["candidate"].isna()][
        ["candidate", "pact", "party", "district", "votes"]
    ]

    # concatenate to main dataframe
    results = pd.concat((results, district_df), axis=0)
    
    
    # EXTRA: load percentage of completion for each district
    workbook = load_workbook(filename=f"datos_2025/distrito_{n}.xlsx")
    sheet = workbook.active
    status = sheet["A9"].value.replace(".", "")
    nums = re.findall(r'\d+', status)
    checked = int(nums[0])
    total = int(nums[1])
    
    checked_tables_districts.append(checked)
    total_tables_districts.append(total)
    percentage_tables_districts.append(checked / total)
    

# reset index, drop null candidates (edge case)
results = results.reset_index().iloc[:, 1:]
results = results.drop(results[results["candidate"].isna()].index)

# give independents their own pct (workaround)
results.loc[results["pact"]=="CANDIDATURA INDEPENDIENTE", "pact"] = (
    results[results["pact"]=="CANDIDATURA INDEPENDIENTE"]["candidate"]
)

# cast district number and vote tallies to integers
results[["district", "votes"]] = results[["district", "votes"]].astype(int)

# clean names
results["party"] = results["party"].apply(clean_party)
results["candidate"] = results["candidate"].apply(clean_candidate)

# add percentage of votes (of their district) to each candidate
results = add_percentage(results)

In [6]:
# rename parties and pacts
results = results.replace({
    "pact": {
        "A - PARTIDO ECOLOGISTA VERDE": "PEV",
        "B - VERDES, REGIONALISTAS Y HUMANISTAS": "FREVS/AH",
        "C - UNIDAD POR CHILE": "UNIDAD",
        "D - IZQUIERDA ECOLOGISTA POPULAR ANIMALISTA Y HUMANISTA": "PH/PI",
        "E - MOVIMIENTO AMARILLOS POR CHILE": "AMARILLOS",
        "F - PARTIDO DE TRABAJADORES REVOLUCIONARIOS": "PTR",
        "G - PARTIDO ALIANZA VERDE POPULAR": "PAVP",
        "H - POPULAR": "POPULAR",
        "I - PARTIDO DE LA GENTE": "PDG",
        "J - CHILE GRANDE Y UNIDO": "CHILEVAMOS",
        "K - CAMBIO POR CHILE": "REP/PNL/PSC",
        "CANDIDATURA INDEPENDIENTE": "INDEP"
    }
})

In [7]:
# most and least voted candidates
results.sort_values("votes", ascending=False)

,candidate,pact,party,district,votes,percentage
123,DANIEL MANOUCHEHRI MOGHADAM KASHAN LOBOS,UNIDAD,PS,5,94501,0.211611
517,PAMELA JILES MORENO,PDG,PDG,12,94478,0.152701
278,GUSTAVO GATICA VILLARROEL,UNIDAD,PCCH,8,94444,0.123144
381,GONZALO WINTER ETCHEBERRY,UNIDAD,FA,10,94085,0.145329
50,SEBASTIAN PATRICIO VIDELA CASTILLO,UNIDAD,PL,3,74340,0.233065
...,...,...,...,...,...,...
406,YAMILA DE LOURDES MARTINEZ URRUTIA,PTR,PTR,10,430,0.000664
95,MARCELO ISMAEL RUIZ TAGLE ESCOBAR,POPULAR,POPULAR,4,419,0.002431
239,FRANCISCO-JESUS PEÑALOZA AGUILERA,POPULAR,POPULAR,7,399,0.000712
241,MARIELA DEL CARMEN HIDALGO DIAZ,POPULAR,POPULAR,7,377,0.000673


In [8]:
len(results[results["candidate"].duplicated()])

0

In [9]:
# save data to csv file
results.to_csv("datos_2025.csv")

pd.DataFrame(
    {
        "checked": checked_tables_districts,
        "total": total_tables_districts,
        "percentage": percentage_tables_districts,
    },
    index=range(1, 29)
).to_csv("mesas.csv")